## Dependencies

---

In [2]:
import pandas as pd
# %run "./Extraction.ipynb"

## Transformation

---

### Functions:

In [ ]:
def replace_datapoint(value:str, replacement:any, datalist: list[any]) -> None:
    """
    Replace a datapoint if we know the value of datapoint to replace. Useful for kilometers list bug.
    :param value: value of datapoint to remove
    :param replacement: value of datapoint to insert
    :param datalist: list where the operation will be applied
    """
    try:
        to_replace = datalist.index(value)
        datalist[to_replace] = replacement
    except ValueError as ve:
        print("That values doesn't exists")


In [ ]:
def fix_tuples(csv:str, index:int) -> pd.DataFrame:
    """
    Fix displaced tuples in csv file.
    :param csv: name of file to fix (without extension).
    :param index: index of registry in description column to delete.
    :return: dataframe with tuples fixed.
    """
    dataframe = pd.read_csv(f"persistence/{csv}.csv")

    models = list(dataframe["model"])
    year = list(dataframe["year"])
    kilometer =  list(dataframe["kilometers"])
    engine = list(dataframe["engine"])
    price = list(dataframe["price"])
    descriptions = list(dataframe["description"])

    lists = [models, year, kilometer, engine, price]
    lists = [data_list.pop(0) for data_list in lists]
    descriptions.pop(index)

    return pd.DataFrame({
        "model": models,
        "year": year,
        "kilometers": kilometer,
        "engine": engine,
        "price": price,
        "description": descriptions
    })


### Messy datapoints:

 Since some ads don't have their fields complete, the scraper skip to the next field extracting for example the `engine` value inside `kilometers`, due kilometers field was empty and was skipped.

In [ ]:
frame = pd.read_csv("persistence/April-08.csv")

### Displaced tuples:

Since some ads structure and the way that data was extracted happens a displacement for one ad that wasn't extracted as it should be, so I've to remove some unusable datapoins to keep the coherence between `model` and `description`, due the unwanted behavior affect the `description` values.

In [ ]:
frame = fix_tuples("April-08", 33)

In [ ]:
frame.tail()

In [ ]:
frame.to_csv("persistence/March-21.csv", index=False)